Using ___DOI___ info, Scopus database will be used to retrieve article info such as `scopus_id`

01-03-2017.

---

In [38]:
from pyscopus import Scopus
apikey = 'ecce3ba70f145bb72f4521013930868a'
scopus = Scopus(apikey)

In [3]:
import pandas as pd

In [4]:
doi_arr = pd.np.loadtxt('./JCDL-DOI-01-16.csv', dtype=str)
doi_arr

array(['10.1145/379437.379439', '10.1145/379437.379440',
       '10.1145/379437.379441', ..., '10.1145/2756406.2756930',
       '10.1145/2756406.2756932', '10.1145/2756406.2756934'],
      dtype='<U23')

In [36]:
','.join(doi_arr[:10])

'10.1145/379437.379439,10.1145/379437.379440,10.1145/379437.379441,10.1145/379437.379442,10.1145/379437.379443,10.1145/379437.379444,10.1145/379437.379445,10.1145/379437.379446,10.1145/379437.379447,10.1145/379437.379448'

In [19]:
import requests

---

#### Search by DOI

##### Test for one DOI

In [5]:
i = 0
doi = doi_arr[i]
doi

'10.1145/379437.379439'

In [6]:
search_df = scopus.search("DOI(%s)"%doi, count=1)
search_df

,affiliation,aggregation_type,citation_count,cover_date,doi,eissn,isbn,issn,page_range,publication_name,scopus_id,subtype_description,title,volume
0,"[{'name': 'Technische Universitat Wien', 'city...",Conference Proceeding,9,2001-01-01,10.1145/379437.379439,None,1581133456,None,1-10,Proceedings of the ACM International Conferenc...,84901253569,Conference Paper,Integrating automatic genre analysis into digi...,None


##### Apply

In [8]:
pub_info_df = [scopus.search("DOI(%s)"%doi, count=1) for doi in doi_arr]
len(pub_info_df), doi_arr.size

(1376, 1376)

In [14]:
pub_info_df = pd.concat(pub_info_df, ignore_index=True)

In [15]:
pub_info_df.shape

(977, 14)

In [55]:
pub_info_df.cover_date.str.slice(0, 4).value_counts().sort_index()

2001    105
2006    108
2007    117
2008    110
2009    110
2010     66
2011    102
2012     99
2013     97
2015     63
Name: cover_date, dtype: int64

In [16]:
pub_info_df.to_csv('./JCDL-meta-01-16.csv', index=False)

---

#### Full text??

Try to see if full text is avaialbel for the given DOIs

##### Test for one DOI

In [31]:
doi = '10.1145/379437.379440'
doi

'10.1145/379437.379440'

In [32]:
fulltext_api_url = 'https://api.elsevier.com/content/article/doi/' + doi
fulltext_api_url

'https://api.elsevier.com/content/article/doi/10.1145/379437.379440'

In [33]:
r = requests.get(fulltext_api_url, params={'apikey': '94ad3b792ed6fe7680e2eb0f08887379'})

In [34]:
r.url

'https://api.elsevier.com/content/article/doi/10.1145/379437.379440?apikey=94ad3b792ed6fe7680e2eb0f08887379'

In [37]:
print(r.content)

b'<service-error><status><statusCode>RESOURCE_NOT_FOUND</statusCode><statusText>The resource specified cannot be found.</statusText></status></service-error>'


Seems not working

---

#### Abstract

In [41]:
info_df = pd.read_csv('./JCDL-meta-01-16.csv',
                      converters={'scopus_id': str})
info_df.head(2)

,affiliation,aggregation_type,citation_count,cover_date,doi,eissn,isbn,issn,page_range,publication_name,scopus_id,subtype_description,title,volume
0,"[{'name': 'Technische Universitat Wien', 'city...",Conference Proceeding,9,2001-01-01,10.1145/379437.379439,NaN,1581133456,NaN,1-10,Proceedings of the ACM International Conferenc...,84901253569,Conference Paper,Integrating automatic genre analysis into digi...,NaN
1,"[{'name': 'Universita degli Studi di Firenze',...",Conference Proceeding,2,2001-01-01,10.1145/379437.379440,NaN,1581133456,NaN,11-20,Proceedings of the ACM International Conferenc...,84901254140,Conference Paper,Text categorization for multi-page documents: ...,NaN


##### Test for one article

In [42]:
scopus_id = info_df.loc[0, 'scopus_id']
scopus_id

'84901253569'

In [43]:
scopus.retrieve_abstract(scopus_id)

{'abstract': 'With the number and types of documents in digital library systems increasing, tools for automatically organizing and presenting the content have to be found. While many approaches focus on topic-based organization and structuring, hardly any system incorporates automatic structural analysis and representation. Yet, genre information (unconsciously) forms one of the most distinguishing features in conventional libraries and in information searches. In this paper we present an approach to automatically analyze the structure of documents and to integrate this information into an automatically created content-based organization. In the resulting visualization, documents on similar topics, yet representing different genres, are depicted as books in differing colors. This representation supports users intuitively in locating relevant information presented in a relevant form. Copyright 2001 ACM.',
 'citedby-count': '9',
 'dc:publisher': 'Association for Computing Machinery acmhe

##### Apply

In [44]:
abstract_df = list()
not_working_list = list()
for scopus_id in info_df.scopus_id.values:
    try:
        abstract_df.append(scopus.retrieve_abstract(scopus_id))
    except:
        not_working_list.append(scopus_id)
        continue
len(abstract_df)

931

In [47]:
abstract_df[0]

{'abstract': 'With the number and types of documents in digital library systems increasing, tools for automatically organizing and presenting the content have to be found. While many approaches focus on topic-based organization and structuring, hardly any system incorporates automatic structural analysis and representation. Yet, genre information (unconsciously) forms one of the most distinguishing features in conventional libraries and in information searches. In this paper we present an approach to automatically analyze the structure of documents and to integrate this information into an automatically created content-based organization. In the resulting visualization, documents on similar topics, yet representing different genres, are depicted as books in differing colors. This representation supports users intuitively in locating relevant information presented in a relevant form. Copyright 2001 ACM.',
 'citedby-count': '9',
 'dc:publisher': 'Association for Computing Machinery acmhe

In [52]:
abstract_df = pd.DataFrame(abstract_df)
abstract_df.head(2)

,abstract,citedby-count,dc:publisher,eid,prism:aggregationType,prism:coverDate,prism:doi,prism:endingPage,prism:isbn,prism:issn,prism:pageRange,prism:publicationName,prism:startingPage,prism:url,prism:volume,scopus-id,source-id,srctype,title
0,With the number and types of documents in digi...,9,Association for Computing Machinery acmhelpacm...,2-s2.0-84901253569,Conference Proceeding,2001-01-01,10.1145/379437.379439,10,"[{'$': '1581133456'}, {'$': '9781581133455'}]",NaN,1-10,Proceedings of the ACM International Conferenc...,1,http://api.elsevier.com/content/abstract/scopu...,NaN,84901253569,62702,p,Integrating automatic genre analysis into digi...
1,Text categorization is typically formulated as...,2,Association for Computing Machinery acmhelpacm...,2-s2.0-84901254140,Conference Proceeding,2001-01-01,10.1145/379437.379440,20,"[{'$': '1581133456'}, {'$': '9781581133455'}]",NaN,11-20,Proceedings of the ACM International Conferenc...,11,http://api.elsevier.com/content/abstract/scopu...,NaN,84901254140,62702,p,Text categorization for multi-page documents: ...


In [53]:
abstract_df['prism:coverDate'].str.slice(0, 4).value_counts()

2009    106
2007    103
2008    101
2006    101
2011    101
2001     99
2012     96
2013     96
2010     66
2015     62
Name: prism:coverDate, dtype: int64

In [56]:
abstract_df.to_csv('./JCDL-abstract-01-16.csv', index=False)